<h1 align=center style="line-height:300%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پروژه دوم درس داده کاوی
</font>
</h1>
<h3 align=center style="line-height:100%;font-family:vazir;color:#0099cc">
استاد درس: دکتر ناظرفرد
</h3>
<h5 align=center style="font-size: 20px;line-height:100%;font-family:vazir;color:#0099cc">
طراح تمرین عملی: فرهاد امان  
    </h5>

## Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#we will need this imports for preprocessing sections
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder 

In [1]:
#importing models
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor

ModuleNotFoundError: No module named 'xgboost'

## Loading Data to Pandas DataFrame

## Droping Unnecessary Columns

## Splitting Data to Train and Test

## Visualizing Data Distribution and Correlation

## Linear Regression

## Polynomial Regression

## Ridge Regression

## Lasso Regression

## Elastic Net Regression

## XGBoost Regression 

# Assosiation Rules

In [1]:
# 0. Import Libraries


# 1. Data Loading


# 2. Preprocess Data


# 3. Apply TransactionEncoder


# 4. Generate Frequent Itemsets


# 5. Generate Association Rules


# 6. Display Rules

